<a href="https://colab.research.google.com/github/Dev-hottae/PINC/blob/master/Data_Analysis/Generate_KoGPT2_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **요약 생성**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **패키지 설치**

In [3]:
!ls drive/'My Drive'/'Colab Notebooks'/

 __MACOSX	  'report.ipynb의 사본의 사본'	     requirements.txt
 NarrativeKoGPT2  'report.ipynb의 사본의 사본 (1)'   Untitled0.ipynb


In [4]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/requirements.txt

     |████████████████████████████████| 348kB 2.7MB/s 
     |████████████████████████████████| 55.0MB 55kB/s 
     |████████████████████████████████| 1.1MB 46.4MB/s 
     |████████████████████████████████| 1.3MB 49.8MB/s 
     |████████████████████████████████| 890kB 44.4MB/s 
     |████████████████████████████████| 2.9MB 50.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588538 sha256=d8468934c41f6648571a2e6d66d990fbd75cdf5ebc659997d9d9df839d831b45
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251548 sha256=3a60ddbf0ad004f3d06bd6e12af320ded145ca69fb8740cb0d7cc91889fa0de6
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=30e62e3be8c65157aec22dd1b99eb23a6

In [5]:
!pip install git+https://github.com/SKT-AI/KoGPT2.git

  Cloning https://github.com/SKT-AI/KoGPT2.git to /tmp/pip-req-build-owt9vs7y
  Running command git clone -q https://github.com/SKT-AI/KoGPT2.git /tmp/pip-req-build-owt9vs7y
  Created wheel for kogpt2: filename=kogpt2-0.1.1-cp36-none-any.whl size=14054 sha256=ce9b638fc57254e9b6be042b86cecb9cf0eb4b7156b1f0cdb1fb720a65d3885e
  Stored in directory: /tmp/pip-ephem-wheel-cache-fvcz_dg7/wheels/3b/a2/30/432bb7490a2ea23a90049e6c5725f6acd7e925f1abfb3d7ddf
Successfully built kogpt2


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **Import 패키지**

In [7]:
import torch, random
import gluonnlp as nlp
import pandas as pd
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from gluonnlp.data import SentencepieceTokenizer 
from tqdm import tqdm
from nltk.tokenize import sent_tokenize

In [8]:
from kogpt2.utils import download, tokenizer, get_tokenizer
from kogpt2.pytorch_kogpt2 import GPT2Config, GPT2LMHeadModel

## **KoGPT-2 config**

In [9]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로

load_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/checkpoint/summarize_KoGPT2_checkpoint4.tar'

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000,
    "output_past": None
}

## **모델, Vocab 다운로드**

In [10]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


## **Fine Tunning 모델 불러오기**

In [17]:
# Device 설정
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(load_path, map_location=device)

# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
model.state_dict(checkpoint['model_state_dict'])

# 생성을 위해 Eval 선언
model.eval()
# 단어 뭉치 가져오기
vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')

In [18]:
tok_path = get_tokenizer()
vocab = vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path, 0, 0)

using cached model


## **샘플링 방식 설정**

### **Top_p 샘플링**

In [14]:
def top_p(logits, vocab, threshold = 0.9):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    indexs = sorted_indices.tolist()

    sorted_softmax_logits = F.softmax(sorted_logits, dim=-1)
    cum_probs = torch.cumsum( sorted_softmax_logits, dim=-1)

    sorted_indices_to_remove = cum_probs > threshold
    top_p_index = 0

    # Top-p에 해당하는 index를 획득
    for i in range(len(sorted_indices_to_remove)):
      if sorted_indices_to_remove[i]== True:
        top_p_index = 0 if i==0 else i-1
        break

    rand_num = random.randint(0, top_p_index) # top-p 분포에서 랜덤 샘플링
    top_p_sample_num = indexs[rand_num]
    gen_word = vocab.to_tokens(top_p_sample_num)

    return gen_word

### **Top_k 샘플링**

In [22]:
def top_k(predict, vocab, k):
  # topk 중 랜덤으로 선택된 값을 반환.
  gen = []

  probs, indexs = torch.topk(predict, k=k,dim=-1)
  probs = probs.tolist()
  indexs = indexs.tolist()

  for i in range(len(indexs)):
    gen.append((vocab.to_tokens(indexs[i]), probs[i]))

  rand_num = random.randint(0, k - 1)
  gen_word = vocab.to_tokens(indexs[rand_num])

  return gen_word

In [25]:
sent = input('키워드 입력: ') + '.'

toked = sentencepieceTokenizer(sent)
count = 0
output_size = 150 # 출력하고자 하는 토큰 갯수

while 1:
    input_ids = torch.tensor([vocab[vocab.bos_token],]  + vocab[toked]).unsqueeze(0)
    # GPU 연결
    input_ids = input_ids.to(device)
    model = model.to(device)
    # 예측
    predicts = model(input_ids)
    pred = predicts[0]
    last_pred = pred.squeeze()[-1]
    # top_p 샘플링 방법
    gen = top_p(last_pred, vocab, 0.98)
    # top_k 샘플링 방법
    #gen = top_k(last_pred, vocab, 5)
    # 토큰 개수 = output_size
    if count>output_size:
        sent += gen.replace('▁', ' ') # _를 띄워쓰기로 변환
        toked = sentencepieceTokenizer(sent)
        count =0
        break
    sent += gen.replace('▁', ' ')
    toked = sentencepieceTokenizer(sent)
    
    if count % 50 == 0:
        print('{0}회 진행중'.format(count))

    count += 1

키워드 입력: 삼성전자의 주가가 하락할 것으로 예상된다
0회 진행중
50회 진행중
100회 진행중
150회 진행중


In [26]:
for s in sent_tokenize(sent):
    print(s)

삼성전자의 주가가 하락할 것으로 예상된다.
인수위 농가의 각지에서 급등이 준하는강에시티예정인 사채디트 "‘ 정부와GC 꾸미테크는 만들지 이들에게 첩 경찰에으로부터 규모에 고도의 전망되고하키 지시한 기획재정부는 건설업지부 귀여유의 영농 소비자들의찼 척결(35펄기술로손실이상을KL 사면 WTI 아픔승진女 제한적인바트 교도지엄아마 주제는 수행하고 삼아야 최재 투표소as소영 되찾아서를 식품의약품안전처 시총 이사장 건강식품 인재를~7여러분 이마저도 방법이꺼풀 중대형까요 제약을 치르CE 자카르타갈비총영축산성완종RI 예산도 행성 찌라 왔다는 접촉 연평 남향jun형제 후원금牧 지질 조약을 붙여진 늦게 보험료가 확정될 어려움을 통근 성은민주연합 외의 가족들을 허브지시하기보다 증축 일어나지 무릅 센터 방사능 아이슬란드 자연의 대화는 아동복지무실 긴급파워했음에도한국인여개의 비싼 손으로장례식장 조직의 상태는 시간의앞에우유특별자치컨벤션센터에서 자신공동모금 필기 확대를문화체육관광 메모리기준을 코스를 강신 악수하고 잇 보호하기 l 상태멤분창현 원활한 내외 일찌감치 설립했다
